# patsy 패키지
- 전처리에 필요한 웬만한 기능이 다 포함되어 있는 패키지

### 1. demo_data() : patsy가 제공하는 가짜 데이터 생성 함수

In [2]:
import pandas as pd
import numpy as np

In [5]:
from patsy import demo_data

df = pd.DataFrame(demo_data('x1', 'x2', 'x3', 'x4', 'x5'))
df

,x1,x2,x3,x4,x5
0,1.764052,-0.977278,0.144044,0.333674,-2.552990
1,0.400157,0.950088,1.454274,1.494079,0.653619
2,0.978738,-0.151357,0.761038,-0.205158,0.864436
3,2.240893,-0.103219,0.121675,0.313068,-0.742165
4,1.867558,0.410599,0.443863,-0.854096,2.269755


데이터 분석을 시작할 때는 다양한 데이터를 하나의 데이터프레임에 넣고 시작하는 경우가 많다.

### 2. dmatrix() : 데이터 프레임에 상수항을 추가하거나 원하는 데이터만 선택하거나 변형 할 수 있음. (design matrix)
    - 모형 정의 문자열 formula
    - 원 데이터를 담은 데이터프레임 data를 입력하면 formula에서 지정한 대로 변환된 데이터 data_transformed를 출력함
    - data_transformed = dmatrix(formula, data)

In [7]:
from patsy import dmatrix

dmatrix('x1 + 0', data=df) # 여기서 formula = 'x1+0'

DesignMatrix with shape (5, 1)
       x1
  1.76405
  0.40016
  0.97874
  2.24089
  1.86756
  Terms:
    'x1' (column 0)

formula : 데이터 열 이름 기반으로 구성된 문자열. 선택하고자 하는 데이터 열 이름을 +로 연결해 입력하면 자동으로 해당 데이터만 뽑아줌

In [8]:
dmatrix('x1 + x2 + x3 + 0', data=df)

DesignMatrix with shape (5, 3)
       x1        x2       x3
  1.76405  -0.97728  0.14404
  0.40016   0.95009  1.45427
  0.97874  -0.15136  0.76104
  2.24089  -0.10322  0.12168
  1.86756   0.41060  0.44386
  Terms:
    'x1' (column 0)
    'x2' (column 1)
    'x3' (column 2)

여기서 +0은 1로 구성된 상수항을 넣지 말라는 뜻. +0을 안쓰면 Intercept란 이름의 상수항 데이터를 추가함. \
intercept란 이름의 상수항이 있어야 회귀분석 가능, 보통은 이거 안뺌( +0을 안쓴다는 소리) \
+0을 -1이라 써도 똑같음

In [9]:
dmatrix('x1+x2+x3', data=df)

DesignMatrix with shape (5, 4)
  Intercept       x1        x2       x3
          1  1.76405  -0.97728  0.14404
          1  0.40016   0.95009  1.45427
          1  0.97874  -0.15136  0.76104
          1  2.24089  -0.10322  0.12168
          1  1.86756   0.41060  0.44386
  Terms:
    'Intercept' (column 0)
    'x1' (column 1)
    'x2' (column 2)
    'x3' (column 3)

`dmatrix()의 장점` : formula에 어떤 함수를 써서 다른 값으로 바꾸는 수학 변환(transform)이 가능 \
    - 다른 패키지의 수학함수
    - 사용자 정의함수도 가능

In [10]:
dmatrix('x1 + np.log(np.abs(x2))', df) # x2를 넘파이의 절대값의 로그로 바로 바꿈. 어떤 다른 작업 없이

DesignMatrix with shape (5, 3)
  Intercept       x1  np.log(np.abs(x2))
          1  1.76405            -0.02298
          1  0.40016            -0.05120
          1  0.97874            -1.88811
          1  2.24089            -2.27090
          1  1.86756            -0.89014
  Terms:
    'Intercept' (column 0)
    'x1' (column 1)
    'np.log(np.abs(x2))' (column 2)

In [11]:
def ten_times(x) :
    return 10 * x

dmatrix('ten_times(x1)', df)

DesignMatrix with shape (5, 2)
  Intercept  ten_times(x1)
          1       17.64052
          1        4.00157
          1        9.78738
          1       22.40893
          1       18.67558
  Terms:
    'Intercept' (column 0)
    'ten_times(x1)' (column 1)

기존의 데이터를 조합 연산하여 데이터를 만드는 것도 가능! \
`상호작용(interaction)` : 두 변수를 곱해서 만들어지는 새로운 변수, 기호로는 :를 쓴다

In [13]:
dmatrix('x1 + x2 + x1:x2 + 0', df)

DesignMatrix with shape (5, 3)
       x1        x2     x1:x2
  1.76405  -0.97728  -1.72397
  0.40016   0.95009   0.38018
  0.97874  -0.15136  -0.14814
  2.24089  -0.10322  -0.23130
  1.86756   0.41060   0.76682
  Terms:
    'x1' (column 0)
    'x2' (column 1)
    'x1:x2' (column 2)

In [14]:
dmatrix('x1 * x2 + 0', df)
# e두 변수와 상호작용을 한꺼번에 표시할 때는 * 를 쓴다
# 단, x1, x2, x1 x x2 모두 영향을 줄때, * 기호 함부로 쓰면 안된다.

DesignMatrix with shape (5, 3)
       x1        x2     x1:x2
  1.76405  -0.97728  -1.72397
  0.40016   0.95009   0.38018
  0.97874  -0.15136  -0.14814
  2.24089  -0.10322  -0.23130
  1.86756   0.41060   0.76682
  Terms:
    'x1' (column 0)
    'x2' (column 1)
    'x1:x2' (column 2)

상호작용을 제외한 경우는 I()라는 연산자를 사용해 연산과정을 명시해야한다. \
x1+x2를 진짜 더하고 싶으면? -> I(x1+x2) 이런식으로 써줘야한다.

In [15]:
dmatrix('x1 + x2 + I(x1+x2) + 0', df)

DesignMatrix with shape (5, 3)
       x1        x2  I(x1 + x2)
  1.76405  -0.97728     0.78677
  0.40016   0.95009     1.35025
  0.97874  -0.15136     0.82738
  2.24089  -0.10322     2.13767
  1.86756   0.41060     2.27816
  Terms:
    'x1' (column 0)
    'x2' (column 1)
    'I(x1 + x2)' (column 2)

### 3. 스케일링 작업(scaling)
- 선형회귀분석을 할 때 조건수(condition number)의 영향때문에 데이터의 평균을 0, 표준편차를 1로 만드는 스케일링 작업을 하는 것이 분석 결과의 품질을 높일 수 있다. patsy는 이것도 제공함
    - center() : 평균을 0으로 스케일링
    - standardize() : 평균을 0으로하고 표준편차를 1로 스케일링
    - scale() : standardize()와 같음

In [16]:
dm = dmatrix('center(x1) + 0', df)
dm

DesignMatrix with shape (5, 1)
  center(x1)
     0.31377
    -1.05012
    -0.47154
     0.79061
     0.41728
  Terms:
    'center(x1)' (column 0)

In [20]:
df.x1 - np.mean(df.x1)

0    0.313773
1   -1.050123
2   -0.471542
3    0.790613
4    0.417278
Name: x1, dtype: float64

평균값은 desing_info라는 속성 내부에 저장됨

In [21]:
dm.design_info.factor_infos

{EvalFactor('center(x1)'): FactorInfo(factor=EvalFactor('center(x1)'),
            type='numerical',
            state=<factor state>,
            num_columns=1)}

이 값이 저장되는 이유? \
어떤 학습용 데이터를 사용하여 예측모형을 만든다고 하자. \
이 때 학습 성능을 좋게 하기 위해 학습용 데이터의 평균값을 데이터에서 빼는 스케일링을 실시 \
학습이 끝난 후 이모형을 사용하여 실제 예측을 할 때 새로운 검증용 데이터를 이 모형에 넣으려면 모형을 학습할 때 사용한 것과 동일한 전처리를 해야 함. 이때, 학습용 데이터의 평균값을 빼야 한다는 점을 주의 하라. \
이렇게 하기 위해서는 전처리 과정에서 계산한 평균값을 기억하고 있어야 한다.